# Tạo recovery/root key/address key với cardano-address



Bảng chú giải thuật ngũ:
- x = private
- s = signing
- v = verification
- policy.script: là điều kiện của policy tạo token
- policyID: là hash của policy.script 
- assetID = hash policyID + asset name
- assetname trong môi trường mã máy cardano biểu diễn ở dạng hexcode


Tạo recovery phrase


In [ ]:
cardano-address recovery-phrase generate \
        --size 12 > Demo/phrase1.prv 
echo "Bộ từ bí mật là: $(cat Demo/phrase1.prv)"
#Các size tiêu chuẩn Cardano 9,12,15,18,21,24
#Thêm passphrase-spending password
#cardano-address key from-recovery-phrase Shelley --passphrase from-hex <Demo/phrase1.prv>
#cardano-address key from-recovery-phrase Shelley --passphrase from-mnemonic <Demo/phrase1.prv>


Tạo Root key - Master key 

In [ ]:
cardano-address key from-recovery-phrase Shelley < Demo/phrase1.prv > Demo/root.xsk 
#Hệ thống sẽ tạo ra file root.xsk
echo $(cat Demo/root.xsk)
         

Tạo key cho address và address

In [ ]:
cardano-address key child 1855H/1815H/0H/0/0 < Demo/root.xsk > Demo/key0.xsk
cardano-address key child 1855H/1815H/0H/0/0 < Demo/root.xsk | cardano-address key public --with-chain-code > Demo/key0.xvk
echo $(cat Demo/key0.xsk)
cardano-cli key convert-cardano-address-key --shelley-payment-key --signing-key-file Demo/key0.xsk --out-file Demo/key0.skey
cardano-cli key verification-key --signing-key-file Demo/key0.skey --verification-key-file Demo/key0.vkey

Tạo address từ vkey

In [ ]:
#cardano-cli address build \
#--payment-verification-key-file key0.vkey \
#--stake-verification-key-file stake.vkey \
#--out-file payment0.addr \
#--testnet-magic $CARDANO_NODE_MAGIC 

#hoặc dùng lệnh từ cardano-address như nhau
#cardano-address key hash < Demo/addr0.xvk
cardano-address address payment --network-tag testnet  < Demo/key0.xvk > Demo/payment.addr 


Tạo policy key & policy hash


In [ ]:
cardano-cli address key-gen \
    --verification-key-file policy0.vkey \
    --signing-key-file policy0.skey
#cardano-cli address key-hash --payment-verification-key-file policy0.vkey 
#echo $(cardano-cli address key-hash --payment-verification-key-file policy0.vkey)


Tạo policy script & policy ID & tạo token name

In [64]:
#touch Demo/policy0.script
#echo "{\"type\": \"sig\", \"keyHash\" : \"c34dc53aec21f3f339f943148276c6d9f08fab4b0f21fee2f23a2509\" }" > Demo/policy0.script
#cardano-cli transaction policyid --script-file Demo/policy0.script > Demo/policyid 
#echo $(cat Demo/policy0.script)
#echo -n "vietcore" | xxd -ps > 
#namehex #hoặc search tool nào đó để đổi tên từ text qua hexcode

## Tạo giao dịch mint


#Lấy tADA testnet hoặc nếu mainnet thì phải có ADA ở đâu đó
https://docs.cardano.org/cardano-testnet/tools/faucet/

In [1]:
MINT=$(cat Demo/payment.addr)
cardano-cli query utxo \
    --address $MINT \
    --testnet-magic $CARDANO_NODE_MAGIC

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------


In [ ]:
#có thể chuẩn bị gán biến cho nó đơn giản
#TX_HASH="instert TxHash here"
#TX_IX="insert TxIx here"
#TX_AMOUNT="9899668558"
#TOKEN_NAME="54657374746F6B656E"
#TOKEN_AMOUNT="10000000"

cardano-cli transaction build-raw \
 --fee $FEE \
 --tx-in $TX_HASH#$TX_IX \
 --tx-out $MY_ADDRESS+$REMAINING+"$TOKEN_AMOUNT $POLICY_ID.$TOKEN_NAME" \
 --mint "$TOKEN_AMOUNT $POLICY_ID.$TOKEN_NAME" \
 --minting-script-file policy0.script \
 --out-file mint.raw

Ký và gửi giao dịch

In [ ]:
cardano-cli transaction sign  \
    --signing-key-file my_address.skey  \
    --signing-key-file asset_policy.skey  \
    --testnet-magic $CARDANO_NODE_MAGIC \
    --tx-body-file mint.raw  \
    --out-file mint.signed

#cardano-cli transaction submit \
#    --tx-file mint.signed \
#    --testnet-magic $CARDANO_NODE_MAGIC

#cardano-cli query utxo \
#    --address $MINT \
#    --testnet-magic $CARDANO_NODE_MAGIC

## Mint NFT


Nếu muốn tách biệt address nhận và tạo token thì tạo address mới, skey mới, policy.skey mới.

In [1]:
#1 dạng script có điều kiện phức tạp hơn ở ví dụ tạo token ở trên. 
# - multi sig
# - thời gian khóa không được mint/burn token
echo "{" >> policy/policy.script
echo "  \"type\": \"all\"," >> policy/policy.script
echo "  \"scripts\":" >> policy/policy.script
echo "  [" >> policy/policy.script
echo "   {" >> policy/policy.script
echo "     \"type\": \"before\"," >> policy/policy.script
echo "     \"slot\": $(expr $(cardano-cli query tip --testnet-magic 2 | jq .slot?) + 10000)" >> policy/policy.script
echo "   }," >> policy/policy.script
echo "   {" >> policy/policy.script
echo "     \"type\": \"sig\"," >> policy/policy.script
echo "     \"keyHash\": \"$(cardano-cli address key-hash --payment-verification-key-file policy/policy.vkey)\"" >> policy/policy.script
echo "   }" >> policy/policy.script
echo "  ]" >> policy/policy.script
echo "}" >> policy/policy.script


In [ ]:
#Tạo metadata file
echo "{" >> metadata.json
echo "  \"721\": {" >> metadata.json
echo "    \"$(cat policy/policyID)\": {" >> metadata.json
echo "      \"$(echo $realtokenname)\": {" >> metadata.json
echo "        \"description\": \"This is my first NFT thanks to the Cardano foundation\"," >> metadata.json
echo "        \"name\": \"Cardano foundation NFT guide token\"," >> metadata.json
echo "        \"id\": \"1\"," >> metadata.json
echo "        \"image\": \"ipfs://$(echo $ipfs_hash)\"" >> metadata.json
echo "      }" >> metadata.json
echo "    }" >> metadata.json
echo "  }" >> metadata.json
echo "}" >> metadata.json

In [ ]:
#Tạo giao dịch mint
cardano-cli transaction build \
--testnet-magic 2 \
--alonzo-era \
--tx-in $txhash#$txix \
--tx-out $address+$output+"$tokenamount $policyid.$tokenname" \
--change-address $address \
--mint="$tokenamount $policyid.$tokenname" \
--minting-script-file $script \
--metadata-json-file metadata.json  \
--invalid-hereafter $slotnumber \
--witness-override 2 \
--out-file matx.raw